# 🤖 Multilayer Perceptron

Esta Jupyter Notebook implementa una MLP para detectar patrones en una matriz 10x10.
Se genera un dataset para luego entrenar el modelo y validarlo.
Se evalúa la precisión para reiterar sobre el modelo con el objetivo de mejorar su eficacia.
Finalmente se exporta el modelo para ser utilizado en la aplicación web.


## 📋 Situación

En una matriz 10x10 se debe poder detectar las letras `b`, `d`, `f` que se corresponden a los siguientes patrones:

![Patrones que el MLP deberá reconocer](assets/patterns.png)

Ante un dato nuevo, el MLP deberá ser capaz de clasificar el contenido de esa matriz en uno de los tres patrones.


## 🗃️ Dataset

Conviene generar el conjunto de datos de manera programática.
Los datasets deberán ser representativos a la hora de definir la distribución de los ejemplos de entrenamiento.
Se definen los patrones 'b', 'd', y 'f'.
Luego, se crea una función `generate_sample` que crea un ejemplo de un patrón dado con una distorsión entre 0 y 0.3. 


In [37]:
import numpy as np

# Define the 10x10 patterns for 'b', 'd', and 'f'
PATTERNS = {
    "b": np.array(
        [
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        ],
        dtype=np.uint8,
    ),
    "d": np.array(
        [
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 1, 0, 0, 0, 0, 1, 0, 0],
            [0, 0, 0, 1, 1, 1, 1, 1, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        ],
        dtype=np.uint8,
    ),
    "f": np.array(
        [
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 1, 1, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 1, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 1, 1, 1, 1, 1, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
            [0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
        ],
        dtype=np.uint8,
    ),
}


def generate_sample(pattern: np.ndarray[np.ndarray[int]], noise=0.0):
    """
    Generates a 10x10 matrix based on a given pattern letter ('b', 'd', or 'f'), with optional noise.
    Args:
        pattern (str): One of 'b', 'd', or 'f'.
        noise (float): Proportion of pixels to flip.
    Returns:
        np.ndarray: A 10x10 numpy array with the pattern and noise applied.
    """
    assert pattern in PATTERNS, "Pattern must be one of 'b', 'd', or 'f'"
    assert 0.0 <= noise <= 0.30, "Noise must be between 0.0 and 0.30"

    matrix = PATTERNS[pattern].copy()
    num_pixels = matrix.size
    num_noisy = int(noise * num_pixels)

    if num_noisy > 0:
        # Choose random indices to flip
        indices = np.unravel_index(
            np.random.choice(num_pixels, num_noisy, replace=False), matrix.shape
        )
        # Flip the selected pixels (0->1, 1->0)
        matrix[indices] = 1 - matrix[indices]

    return matrix


print("A 'b' pattern with zero noise:")
print(generate_sample("d", 0.00))
print()
print("A 'b' with 30% noise, meaning 30 out of 100 cells have been randomly flipped:")
print(generate_sample("d", 0.30))

A 'b' pattern with zero noise:
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 0 1 0 0]
 [0 0 0 1 1 1 1 1 0 0]
 [0 0 1 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 1 0 0]
 [0 0 1 0 0 0 0 1 0 0]
 [0 0 0 1 1 1 1 1 0 0]
 [0 0 0 0 0 0 0 0 0 0]]

A 'b' with 30% noise, meaning 30 out of 100 cells have been randomly flipped:
[[0 0 1 0 0 0 0 1 0 0]
 [0 0 0 1 0 1 0 1 1 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 1 0 0 1 0 0]
 [1 0 0 0 1 0 0 1 1 0]
 [0 0 1 1 0 1 0 1 1 1]
 [0 0 0 0 0 0 0 1 1 0]
 [0 1 0 0 0 0 0 0 0 0]
 [0 1 0 0 1 1 1 0 0 0]
 [0 0 1 0 0 1 1 1 1 0]]


Luego se generan 3 datasets que contengan 100, 500 y 1000 ejemplos. 
El 10% serán patrones sin distorsionar y el resto con una distorsión del 1% al 30%.
Se usará una **distribución uniforme** para ubicar el 90% de ejemplos en el rango de distorsión entre 0.01 y 0.30.

Cada dataset será un `pd.DataFrame` de `pandas` que contiene columnas del `0` al `99` (una por cada celda de la matriz) y una columna final `class` que indica la clase del patrón ('b', 'd' o 'f').


In [ ]:
import pandas as pd

# Extender el ancho máximo al mostrar DataFrames para que entren en una sola línea.
pd.options.display.width = 100


def generate_dataset(n_samples: int):
    """
    Generates a dataset of pattern samples.
    The 90% of samples will have noise between 0.01 and 0.30.
    Args:
        n_samples (int): Number of samples to generate.
    Returns:
        pd.DataFrame: with 100 columns for flattened pattern and 1 column 'class' for pattern class.
    """
    columns = [str(i) for i in range(100)] + ["class"]
    df = pd.DataFrame(0, index=np.arange(n_samples), columns=columns)
    df = df.astype({"class": "str"})

    for i in range(n_samples):
        # 10% sin distorsión, 90% con distorsión entre 1% y 30%
        if i < int(0.1 * n_samples):
            noise = 0.0
        else:
            noise = np.random.uniform(0.01, 0.30)

        # Seleccionar un patrón al azar
        pattern = np.random.choice(list(PATTERNS.keys()))

        sample = generate_sample(pattern, noise).flatten()
        df.iloc[i, :100] = sample
        df.loc[i, "class"] = pattern

    return df


df_100 = generate_dataset(100)
df_500 = generate_dataset(500)
df_1000 = generate_dataset(1000)

print("El dataset de 1000 ejemplos es el siguiente:")
print(df_1000)

El dataset de 1000 ejemplos es el siguiente:
     0  1  2  3  4  5  6  7  8  9  ...  91  92  93  94  95  96  97  98  99  class
0    0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      f
1    0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      b
2    0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      d
3    0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      f
4    0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      f
..  .. .. .. .. .. .. .. .. .. ..  ...  ..  ..  ..  ..  ..  ..  ..  ..  ..    ...
995  0  0  0  0  0  0  0  0  0  1  ...   0   0   0   0   0   0   0   0   0      f
996  0  1  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      d
997  0  0  0  0  0  0  0  0  0  0  ...   0   0   0   0   0   0   0   0   0      f
998  0  1  0  0  0  0  0  1  0  0  ...   0   0   0   0   0   0   0   0   0      d
999  0  0  1  0  0  0  0  0  0  0  ...   0   0   0   

Por cada dataset deberán construirse tres conjuntos de validación con 10%, 20% y 30% de los ejemplos.


In [ ]:
VALIDATION_SPLITS = [0.1, 0.2, 0.3]

# TODO: ...